In [12]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go
import settrade_v2
from settrade_v2 import Investor
import time

In [13]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

WARNING	cassandra.cluster:cluster.py:__init__()- Cluster.__init__ called with contact_points specified, but no load_balancing_policy. In the next major version, this will raise an error; please specify a load-balancing policy. (contact_points = ['127.0.0.1'], lbp = None)
WARNING	cassandra.cluster:cluster.py:protocol_downgrade()- Downgrading core protocol version from 66 to 65 for 127.0.0.1:9042. To avoid this, it is best practice to explicitly set Cluster(protocol_version) to the version supported by your cluster. http://datastax.github.io/python-driver/api/cassandra/cluster.html#cassandra.cluster.Cluster.protocol_version
WARNING	cassandra.cluster:cluster.py:protocol_downgrade()- Downgrading core protocol version from 65 to 5 for 127.0.0.1:9042. To avoid this, it is best practice to explicitly set Cluster(protocol_version) to the version supported by your cluster. http://datastax.github.io/python-driver/api/cassandra/cluster.html#cassandra.cluster.Cluster.protocol_version


WARNING	Thread(ThreadPoolExecutor-1_0) cassandra.cluster:cluster.py:on_down()- Host 127.0.0.1:9042 has been marked down
WARNING	Thread(ThreadPoolExecutor-1_0) cassandra.pool:pool.py:on_exception()- Error attempting to reconnect to 127.0.0.1:9042, scheduling retry in 2.3 seconds: [Errno 61] Tried connecting to [('127.0.0.1', 9042)]. Last error: Connection refused
WARNING	Thread(ThreadPoolExecutor-1_0) cassandra.pool:pool.py:on_exception()- Error attempting to reconnect to 127.0.0.1:9042, scheduling retry in 8.48 seconds: [Errno 61] Tried connecting to [('127.0.0.1', 9042)]. Last error: Connection refused
WARNING	Thread(ThreadPoolExecutor-1_0) cassandra.pool:pool.py:on_exception()- Error attempting to reconnect to 127.0.0.1:9042, scheduling retry in 35.84 seconds: [Errno 61] Tried connecting to [('127.0.0.1', 9042)]. Last error: Connection refused


In [14]:
import settrade_v2
from settrade_v2 import Investor

investor = Investor(
    app_id="atBqrBrphDcChF32",                                 
    app_secret="AK0WsjfbWrRMIL9aCi8b7i12hGdEojdohbUCCb++jDTe", 
    broker_id="SANDBOX",
    app_code="SANDBOX",
    is_auto_queue=False
)

deri = investor.Derivatives(account_no="Man-D")            

account_info = deri.get_account_info()

account_info    

INFO	settradev2:logger.py:wrapped()- [Elapsed time 0.449018s]login - success
INFO	settradev2:logger.py:wrapped()- [Elapsed time 0.710560s]get_account_info - success


{'creditLine': 2000000000.0,
 'excessEquity': 2000000000.0,
 'cashBalance': 2000000000.0,
 'equity': 2000000000.0,
 'totalMR': 0.0,
 'totalMM': 0.0,
 'totalFM': 0.0,
 'callForceFlag': 'No',
 'callForceMargin': 0.0,
 'liquidationValue': 2000000000.0,
 'depositWithdrawal': 0.0,
 'callForceMarginMM': 0.0,
 'initialMargin': 0.0,
 'closingMethod': 'Auto Net'}

In [15]:
#Get Candlestick
market = investor.MarketData()
res = market.get_candlestick(
symbol="AOT",
interval="1d",
limit=1,
normalized=True,)

INFO	settradev2:logger.py:wrapped()- [Elapsed time 0.564961s]get_candlestick - success


In [16]:
if isinstance(res, dict) and "data" in res:
    raw_data = res["data"]
else:
    raw_data = res  # กรณี res เป็น list เลย
df = pd.DataFrame(raw_data)
df

,lastSequence,time,open,high,low,close,volume,value
0,0,1762448400,40.0,41.25,39.5,41.0,28954296,0


In [17]:
session.set_keyspace('stock_data')

In [18]:
# ✅ โหลดรายชื่อหุ้นทั้งหมด
import pandas as pd

try:
    symbols_df = pd.read_excel("//Users/hilmanyusoh/Desktop/Stock-market-chart-pattern-recognition-using-deep-learning/data_collection/settrade1.xlsx")
    symbols = symbols_df['หลักทรัพย์'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ 'get_stock_name' กรุณาตรวจสอบชื่อไฟล์และตำแหน่งให้ถูกต้อง")
    raise  # หยุดโปรแกรมทันทีโดยแสดง error


# ✅ ฟังก์ชันดึง candlestick จาก API
def get_candlestick_data(symbol):
    try:
        res = market.get_candlestick(  # ✅ ต้องกำหนด market ก่อนใช้
            symbol=symbol,
            interval="1d",
            limit=1000,
            normalized=True
        )
        return res
    except Exception as e:
        print(f"❌ ไม่สามารถดึงข้อมูลของ {symbol}: {e}")
        return None

# ✅ ฟังก์ชัน insert ข้อมูลลง Cassandra
def insert_historical_data(symbol, res):
    if not res or 'time' not in res:
        print(f"⚠️ ไม่มีข้อมูลสำหรับ {symbol}")
        return

    for i in range(len(res["time"])):
        # ✅ แปลง timestamp ให้ถูกต้อง
        timestamp = datetime.fromtimestamp(res["time"][i])

        session.execute("""
            INSERT INTO candlestick_data (
                symbol, time, open_price, high_price, low_price,
                close_price, volume, value
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            symbol,
            timestamp,
            res["open"][i],
            res["high"][i],
            res["low"][i],
            res["close"][i],
            res["volume"][i],
            res["value"][i]
        ))

    print(f"✅ เพิ่มข้อมูล {len(res['time'])} รายการของหุ้น {symbol} สำเร็จ!")

# ✅ ดึงและบันทึกข้อมูลทุกหุ้น
for symbol in symbols:
    print(f"\n📦 กำลังโหลดข้อมูล {symbol} ...")
    res = get_candlestick_data(symbol)
    insert_historical_data(symbol, res)
    time.sleep(2)  # 🕒 ลดโอกาสถูก block จาก API

print("\n🎉 เสร็จสิ้นการโหลดข้อมูลทั้งหมด!")


📦 กำลังโหลดข้อมูล 24CS ...


INFO	settradev2:logger.py:wrapped()- [Elapsed time 0.247956s]get_candlestick - success


✅ เพิ่มข้อมูล 1 รายการของหุ้น 24CS สำเร็จ!

📦 กำลังโหลดข้อมูล 2S ...


INFO	settradev2:logger.py:wrapped()- [Elapsed time 0.777282s]get_candlestick - success


✅ เพิ่มข้อมูล 731 รายการของหุ้น 2S สำเร็จ!

📦 กำลังโหลดข้อมูล 3BBIF ...


...

✅ เพิ่มข้อมูล 726 รายการของหุ้น WGE สำเร็จ!

📦 กำลังโหลดข้อมูล WHA ...


INFO	settradev2:logger.py:wrapped()- [Elapsed time 0.450336s]get_candlestick - success


✅ เพิ่มข้อมูล 731 รายการของหุ้น WHA สำเร็จ!

📦 กำลังโหลดข้อมูล WHABT ...


INFO	settradev2:logger.py:wrapped()- [Elapsed time 0.393177s]get_candlestick - success


KeyboardInterrupt: 

In [19]:
stock = "CPALL"
rows = session.execute(f"SELECT * FROM candlestick_data WHERE symbol = '{stock}'")

In [20]:
df = pd.DataFrame(rows, columns=[
    'symbol', 'time', 'open_price', 'high_price', 'low_price', 'close_price', 'volume', 'value'
])

# ✅ แปลงชนิดเวลา + เรียงลำดับ
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values('time').reset_index(drop=True)
df.head()

,symbol,time,open_price,high_price,low_price,close_price,volume,value
0,CPALL,2022-10-17,57.0,57.00,54.50,55.25,0.0,55680175
1,CPALL,2022-10-18,58.5,58.75,57.25,57.25,0.0,55692311
2,CPALL,2022-10-19,58.5,58.50,58.00,58.25,0.0,21308295
3,CPALL,2022-10-20,58.5,58.50,57.75,58.25,0.0,23953636
4,CPALL,2022-10-21,58.5,58.50,58.00,58.50,0.0,13169003


In [21]:
import plotly.graph_objects as go

# ✅ สร้างกราฟ Candlestick
fig = go.Figure()

# 🔹 แท่งเทียนราคา
fig.add_trace(go.Candlestick(
    x=df['time'],
    open=df['open_price'],
    high=df['high_price'],
    low=df['low_price'],
    close=df['close_price'],
    name=stock
))

# 🔹 Volume bar (แถบปริมาณการซื้อขาย)
fig.add_trace(go.Bar(
    x=df['time'],
    y=df['volume'],
    marker_color='rgba(128,128,128,0.3)',
    yaxis='y2',
    name='Volume'
))

# ✅ ปรับ layout ให้เหมือน TradingView
fig.update_layout(
    title=f'📈 {stock} - Candlestick Chart with Volume',
    xaxis_rangeslider_visible=False,
    yaxis=dict(title='Price'),
    yaxis2=dict(
        title='Volume',
        overlaying='y',
        side='right',
        showgrid=False
    ),
    height=600,
    template='plotly_white'
)

fig.show()

WARNING	Thread(ThreadPoolExecutor-1_1) cassandra.cluster:cluster.py:_reconnect_internal()- [control connection] Error connecting to 127.0.0.1:9042:
 Traceback (most recent call last):
   File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/cassandra/cluster.py", line 3577, in _reconnect_internal
    return self._try_connect(host)
           ^^^^^^^^^^^^^^^^^^^^^^^
   File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/cassandra/cluster.py", line 3599, in _try_connect
    connection = self._cluster.connection_factory(host.endpoint, is_control_connection=True)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
   File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/cassandra/cluster.py", line 1670, in connection_factory
    return self.connection_class.factory(endpoint, self.connect_timeout, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^